In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import os
import random
import torch
from torch.utils.data import Dataset
import tqdm

In [ ]:
# 加载 BERT 模型和分词器
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
model = AutoModel.from_pretrained('bert-base-chinese')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [ ]:
# 预测
text = "我爱北京天安门"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [ ]:
print(output.last_hidden_state.shape[:])

torch.Size([1, 9, 768])


In [ ]:
class KnowledgeGraphDataset(Dataset):
    def __init__(self, file_path):
        self.file_path = file_path
        self.triples = self.load_triples(self.file_path)
        self.entities = self.get_entities(self.triples)
        self.positivetriples=self.loadpositivesample(self.file_path)
        self.negativetriples=self.loadnegativesample(self.file_path,self.entities)
        self.relations = self.get_relations(self.triples)
    def loadpositivesample(self,file_path):
        with open(file_path,'r',encoding='utf-8') as f:
            triples=[]
            for line in f:
                triples.append(line)
        return triples

    def load_triples(self, file_path):
        triples = []
        with open(file_path, 'r') as file:
            for line in file:
                a = line.split(',')
                triples.append(a)
        return triples

    def get_entities(self, triples):
        entities = set()
        for triple in triples:
            entities.add(triple[0])
            entities.add(triple[2])
        return list(entities)

    def get_relations(self, triples):
        relations = set()
        for triple in triples:
            relations.add(triple[1])
        return list(relations)

    def loadnegativesample(self,file_path,entities):
        with open(file_path,'r',encoding='utf-8') as f:
            triples=[]
            for line in f:
                a = line.split(',')
                a[2] = random.choice(entities)
                line = a[0]+','+a[1]+','+a[2]
                triples.append(line)
        return triples

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        positivetriple = self.positivetriples[idx]
        negativetriple = self.negativetriples[idx]
        return positivetriple ,negativetriple

# dataset = KnowledgeGraphDataset('triples.txt')
# positive_triple, negative_triples = dataset[0]
# print(f"Positive triple: {positive_triple}")
# print(f"Negative triples: {negative_triples}")

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = torch.nn.Linear(768, 768)
        self.fc2 = torch.nn.Linear(768, 2)
        self.layernorm = torch.nn.LayerNorm(768)
        self.softmax = torch.nn.Softmax()

    def forward(self,cls):
        output = self.fc1(cls)
        output = torch.nn.functional.relu(output)
        output = self.layernorm(output)
        output = self.fc1(output)
        output = self.layernorm(output)
        output = torch.nn.functional.relu(output)
        output = self.layernorm(output)
        output = self.fc1(output)
        output = self.layernorm(output)
        output = torch.nn.functional.relu(output)
        output = self.layernorm(output)
        output = self.fc1(output)
        output = self.layernorm(output)
        output = torch.nn.functional.relu(output)
        output = self.fc2(output)
        output = self.softmax(output)
        return output
# class Model(torch.nn.Module):
#     def __init__(self,triples):
#         super(Model, self).__init__()
#         self.bert = AutoModel.from_pretrained('bert-base-chinese')
#         self.fc = torch.nn.Linear(768, 1)
#         self.triples = triples

#     def forward(self):
#         sentences = [f"{triple[0]} {triple[1]} {triple[2]}" for triple in self.triples]
#         output = self.bert(sentences)
#         cls = output.last_hidden_state[:,0,:]
#         cls.sequeeze(1)
#         output = self.fc(cls)
#         return output


# model=Model(triples)
# output = model.forward()
# print(output.shape[:])

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def trainbertfc(dataset, epoch=1, batch_size=32, learning_rate=0.1):
    model = Model()
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.BCELoss()
    for _ in range(epoch):
        totalloss = 0
        for i in (range(0, len(dataset), batch_size)):
            batch = [dataset[j] for j in range(i, min(i+batch_size, len(dataset)))]
            print(i/len(dataset))
            tokenizer=AutoTokenizer.from_pretrained('bert-base-chinese')
            bert = AutoModel.from_pretrained('bert-base-chinese')
            # batch_labels = []
            # outputlist = []
            for triple in batch:
                pos,neg=triple[0],triple[1]
                # batch_labels.append([1,0])
                # batch_labels.append([0,1])
                pos=','.join(pos)
                neg=','.join(neg)
                posencoded_input = tokenizer(pos, return_tensors='pt')
                negencoded_input = tokenizer(neg, return_tensors='pt')
                posoutput = bert(**posencoded_input)
                negoutput = bert(**negencoded_input)
                cls1=posoutput.pooler_output
                cls2=negoutput.pooler_output
                output1 = model(cls1)
                output2 = model(cls2)
                # outputlist.extend(output1.reshape(1,-1))
                # outputlist.extend(output2.reshape(1,-1))
                # print(outputlist)
            # batch_labels = torch.tensor(batch_labels, dtype=torch.float32,requires_grad=True)
            # outputlist = torch.tensor(outputlist, dtype=torch.float32,requires_grad=True)
                optimizer.zero_grad()
                true_label1 = torch.tensor([1,0], dtype=torch.float32)
                true_label2 = torch.tensor([0,1], dtype=torch.float32)

                loss = criterion(output1.reshape(-1,1).squeeze(), true_label1) + criterion(output2.reshape(-1,1).squeeze(), true_label2)
                #loss = criterion([1,0], output1.reshape(1,-1)) + criterion([0,1],output2.reshape(1,-1))
                loss.backward()
                optimizer.step()

            totalloss += loss.item()
        print(f"Epoch {_+1}/{epoch}, Average Loss: {totalloss/len(dataset):.4f}")
    return model

In [ ]:
file_path = 'train.txt'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = KnowledgeGraphDataset(file_path)
model = trainbertfc(dataset)

0.0


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


0.07459207459207459
0.14918414918414918
0.22377622377622378
0.29836829836829837
0.372960372960373
0.44755244755244755
0.5221445221445221
0.5967365967365967
0.6713286713286714
0.745920745920746
0.8205128205128205
0.8951048951048951
0.9696969696969697
Epoch 1/1, Average Loss: 3.2634


In [ ]:
def testmodel(model, dataset,batch_size=32):
    model.eval()
    for i in (range(0, len(dataset), batch_size)):
        batch = [dataset[j] for j in range(i, min(i+batch_size, len(dataset)))]
        tokenizer=AutoTokenizer.from_pretrained('bert-base-chinese')
        tokenizer=AutoTokenizer.from_pretrained('bert-base-chinese')
        bert = AutoModel.from_pretrained('bert-base-chinese')
        batch_labels = []
        outputlist = []
        for triple in batch:
            pos,neg=triple[0],triple[1]
            batch_labels.append(1)
            batch_labels.append(0)
            pos=','.join(pos)
            # print(pos)
            neg=','.join(neg)
            # print(neg)
            posencoded_input = tokenizer(pos, return_tensors='pt')
            negencoded_input = tokenizer(neg, return_tensors='pt')
            posoutput = bert(**posencoded_input)
            negoutput = bert(**negencoded_input)
            cls1=posoutput.pooler_output
            cls2=negoutput.pooler_output
            output1 = model(cls1)
            output2 = model(cls2)
            outputlist.append(torch.argmax(output1.reshape(-1,1).squeeze(), dim=0))
            outputlist.append(torch.argmax(output2.reshape(-1,1).squeeze(), dim=0))
        batch_labels = torch.tensor(batch_labels, dtype=torch.float32,requires_grad=False)
        outputlist = torch.tensor(outputlist, dtype=torch.float32,requires_grad=False)
        acc = sum((batch_labels==outputlist))
        # distance=torch.nn.PairwiseDistance(p=2)
        # loss = distance(outputlist, batch_labels)
        acc+=acc
    # averagetestloss = totalloss/len(dataset)
    return acc / (len(dataset) * 2)

In [ ]:
file_path = 'test.txt'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = KnowledgeGraphDataset(file_path)
print(testmodel(model,dataset))

tensor(0.1579)
